In [2]:
import pandas as pd
import sys
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
from matplotlib import dates
import seaborn as sns
from dateutil.parser import parse 
from datetime import timedelta, date
%matplotlib inline

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
#print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.3 |Anaconda, Inc.| (default, Oct 13 2017, 12:02:49) 
[GCC 7.2.0]
Pandas version 0.20.3


In [ ]:
train_sample = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/train.csv',                
                         converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
                         parse_dates = ['date'],
                         skiprows = 1,
                         dtype = {'id': 'int32', 'store_nbr': 'int32', 'item_nbr': 'int32', 'onpromotion': 'bool'}, 
                         names = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'])
print (train_sample.head())

## Introduction:
There is always a delicate balance between over-stocking and under-stocking for retailers. Overstocking increases logistics and labor cost, while under-stocking under-feed your customers, and either way harms the profit in long run. The big question to ask in this project is how to predict sales for Favorita, a large supermarket retailer in using historical sales records.  What we have is records of unit sales magnitude for parcular stores, items and at a particular date for the past five years. In addition, there are some categorical features about the stores, items, and if the items/goods are on promotion or perishable. 
    
A plot of total daily sales along all the record peroid indicate that year to year increasing trend as well as variation among years is quite remarkable, it is difficulat to  predict sales a few years and even a few months ahead using historical sales record (come back to this point later).  The first step is to load the most recent two-years record, and to understand the dataset through exploratory analysis.

In [5]:
train_1617 = pd.read_csv('train_1617.csv', parse_dates = ['date'], 
                        dtype = {'id': 'int32', 'store_nbr': 'int8', 'item_nbr': 'int32', 'onpromotion': 'bool'})
print (train_1617.dtypes)
#train_1617 = train_sample.loc[train_sample['date'] > parse('2015-08-15')]
print ('Dimention of 2016-17 training data: {}'.format(train_1617.shape))
print ('Date range of 2016-17 training data: {} - {}'.format(train_1617['date'][:1], train_1617['date'][-1:]))
#train_1617.to_csv('train_1617.csv', index = False)

id                      int32
date           datetime64[ns]
store_nbr                int8
item_nbr                int32
unit_sales            float64
onpromotion              bool
dtype: object
Dimention of 2016-17 training data: (71453276, 6)
Date range of 2016-17 training data: 0   2015-08-16
Name: date, dtype: datetime64[ns] - 71453275   2017-08-15
Name: date, dtype: datetime64[ns]


## Exploratory analysis:
- Daily total sales along time for each store.
- Biweekly pairwise correlation of daily sales for all stores (mean and std).
- Features of stores and items. 
- Time series of continous variables: oil price and holiday.



In [ ]:
store_daily_sum = train_1617.groupby(['date', 'store_nbr'])['unit_sales'].sum()
store_daily_sum = store_daily_sum.add_suffix('').reset_index()
store_daily_sum['date'] = pd.DatetimeIndex(store_daily_sum['date'])
store_daily_sum.head()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, sharex=True, sharey=True, figsize=(25,15))
axes_list = [item for sublist in axes for item in sublist] 

for i in range(15):
    df = store_daily_sum.loc[store_daily_sum['store_nbr'] == str(i+1)]
    axes_list[i].plot(df['date'], df['unit_sales'])
    axes_list[i].set_title(str('store' + df['store_nbr'].unique()))
    axes_list[i].xaxis.set_major_locator(dates.MonthLocator())
    axes_list[i].xaxis.set_major_formatter(dates.DateFormatter('\n%m\n%y'))
    #plt.tight_layout()    

In [ ]:
# is a dataframe storeing daily sales data for each store, it is meant to by grouped by date
def corr_matrix(store_nbr):
    df = store_daily_sum.loc[store_daily_sum['store_nbr'] == str(store_nbr)]
    df['group_label'] = df.groupby(pd.Grouper(key='date', freq='14D', axis=1)).ngroup()
    df['dayofbiweek'] = [i for i in range(1, 15, 1)]*(df.shape[0]//14) + [i for i in range(1, (df.shape[0]%14) + 1, 1)]
    df = df.pivot(index = 'dayofbiweek', columns = 'group_label', values = 'unit_sales')
    return df.corr().as_matrix()

In [ ]:
count = 0
for i in range(52):    
    corr = corr_matrix(store_nbr = str(i+1))
    if corr.shape == (53, 53):
        count+=1
print ('There are {} stores having complete cases of biweek correlation.'.format(count))

corr_all_stores = np.zeros((52, 53, 53))
for i in range(52):
    corr = corr_matrix(store_nbr = str(i+1))
    print 
    if corr.shape == (53, 53):
        corr_all_stores[i::] = corr
    else:
        corr_all_stores[i::] = np.random.uniform(-1, 1, size = (53,53))
print ('Shape of stacks of correlation matrix: {}'.format(corr_all_stores.shape))
corr_mean = np.mean(corr_all_stores, axis = 0)
corr_std = np.std(corr_all_stores, axis = 0)     

In [ ]:
import seaborn as sns
corr = corr_mean
plt.figure(num=None, figsize=(6, 6), dpi=150, facecolor='w', edgecolor='k')
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    ax1 = sns.heatmap(corr, mask = mask, square=True, xticklabels=2)

In [ ]:
import seaborn as sns
plt.figure(num=None, figsize=(6, 6), dpi=150, facecolor='w', edgecolor='k')
corr_mean_rank = corr_mean
corr_mean_rank[corr_mean_rank<0] = 0
final_corr = np.zeros(corr_mean_rank.shape)
for i in range(corr_mean_rank.shape[0]):
    idx = np.argsort(-corr_mean_rank[i, i:])[:10]
    final_corr[np.arange(i, corr_mean_rank.shape[1])[idx], i] = corr_mean_rank[i, i:][idx]    
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    ax2 = sns.heatmap(final_corr, mask = mask, square=True, xticklabels=2)
    


In [ ]:
import seaborn as sns
corr_mean_cutoff = corr_mean
corr_mean_cutoff[corr_mean_cutoff<0.5] = 0
plt.figure(num=None, figsize=(6, 6), dpi=150, facecolor='w', edgecolor='k')   
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    ax1 = sns.heatmap(corr_mean, mask = mask, square=True, xticklabels=2)

### Exploratory analysis for store and item features

In [3]:
stores = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/stores.csv', 
                     dtype = {'store_nbr' : 'int32', 'cluster': 'object'})
items = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/items.csv', 
                    dtype = {'item_nbr' : 'int32', 'family': 'object', 'class': 'object', 'perishable': 'bool'})
oil = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/oil.csv', names = ['date', 'oil_price'], header=0,
                 dtype = {'oil_price': 'float32'}, parse_dates = ['date'])
holidays = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/holidays_events.csv', parse_dates = ['date'])

In [ ]:
print ('dimention of stores = ', stores.shape)
print ('dimention of items = ', items.shape)
print ('dimention of oil = ', oil.shape)
print (stores.head(3))
print (items.head(3))
print (oil.head(3))

### What we learn from exploratory analysis:
- How much data to use based on the biweekly daily sales correlation matrix? 

1) The correlation is only strong for adjacent biweek periods, and sales of Christmas and new year seems correlated with none of the other periods. This raise the limitation of the model, that very suitable for big holiday shopping seasons. Therefore training data including a few months' sale record should serve the purpose.

2) Correlation between months across two years is very weak. Therefore training and validation set will be selected from adjacent biweek periods. 

## Methods
### Data pre-processing

1) Extract store (state/city/cluster) and item (family/type/class) features, as well as time-series variables (i.e. oil price, holiday, day of the week, day of the month).

2) Impute missing data and transform and normalize continouis variables.

3) Label encode categorical variables.


### Feature engineering 
It is worthnoting that so far all the features are not closely related to target variable 'unit_sales', and most of them are categorical variables. In order to precisely predict 'unit_sales', statistical value (mean and std) of target variable over certain time period, can be used to predict target variable. Therefore an additional step to take is calculating statistical values for 'onpromotion' and 'unit_sales'. The intuation behind is to put suits of constrain on the prediction, for example using 3, 7 ,14 days' mean and std of 'unit_sales'of a particular item * store combination to predict the future 'unit_sale' of this item * store combination.

### Assemble all features and develop train, val split
 
train: 2017-07-01 - 2017-07-31;
val: 2017-08-01 - 2017-08-15;
(idelly multiple sets)

### Modeling: 
Random forest and lgb

### Prediction:
Using data from 2017-07-16 to 2017-08-15 to generate prediction for test set posted on kaggle.

In [ ]:
train_17 = train_1617.loc[train_1617['date'] >= parse('2017-07-16')]
train_17 = train_17.set_index(["id"])
train_17.head()

In [ ]:
def prepare_features(df):
    print ('Assembling features ...')
    df['month'] = pd.DatetimeIndex(df['date']).month.astype('object')
    df['day'] = pd.DatetimeIndex(df['date']).day.astype('object')
    df['dayofweek'] = pd.DatetimeIndex(df['date']).dayofweek.astype('object')

    #extract features for the particular date (e.g. oil price, locations and transactions of the store, item features)
    df = df.join(oil.set_index('date'), on = 'date')
    df = df.join(stores.set_index('store_nbr'), on = 'store_nbr')
    df = df.join(items.set_index('item_nbr'), on = 'item_nbr') # extract item features
    
    df = df.join(holidays.set_index('date'), on = 'date', rsuffix = '_h')
    df.loc[df['type_h'].isnull(), 'type_h'] = False
    df.loc[df['type_h'] == 'Holiday', 'type_h'] = True
    df.loc[df['transferred'] == True, 'type_h'] = False

    #Holiday Step1: if it is National holiday, holiday_loc_spec is true 
    df.loc[(df['type_h'] == True) & (df['locale'] == 'National'), 'type_h'] = True

    #Holiday Step2: if it is Regional holiday (state == locale_name), holiday_loc_spec is true 
    df.loc[(df['type_h'] == True) & (df['locale'] == 'Regional') & (df['state'] == df['locale_name']), 
              'type_h'] = True

    #Holiday tep3: if it is Local (locale_name == city), holiday_loc_spec is true 
    df.loc[(df['type_h'] == True) & (df['locale'] == 'Local') & (df['city'] == df['locale_name']), 
         'type_h'] = True
    df['type_h'] = df['type_h'].astype('bool')

    print ('holidays: ', df.loc[df['type_h'] == True].shape[0])
    print ('none-holidays: ', df.loc[df['type_h'] == False].shape[0])

    #Step 4: delete some columns (metadata about holidays)
    df = df.drop(['locale', 'locale_name', 'description', 'transferred'], axis=1)
    
    #Imputing and log transform missing oil data
    from sklearn.preprocessing import Imputer
    from sklearn.preprocessing import StandardScaler
    print ('Imputing daily oil price ...')
    print ('Missing oil price data: {}'.format(np.sum(df['oil_price'].isnull())))
    imputer = Imputer()
    df['oil_price'] = imputer.fit_transform(X = df['oil_price'].values.reshape(-1, 1))
    print ('After imputing missing data: {}'.format(np.sum(df['oil_price'].isnull())))

    df['oil_price_log'] = np.log(df['oil_price'] + 1)
    scaler = StandardScaler()
    df['oil_price_log_norm'] = scaler.fit_transform(X = df['oil_price_log'].values.reshape(-1, 1))
    #print (df['oil_price_log_norm'].describe())
    df = df.drop(['oil_price','oil_price_log'], axis = 1)
    print (df.head(3))
    #print (df.dtypes)
    return df

In [ ]:
train_17 = prepare_features(train_17)
train_17 = train_17.set_index(['store_nbr', 'item_nbr', 'date'])
train_17.head()

In [ ]:
# load test data here
test = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/test.csv', parse_dates = ['date'],
                  dtype = {'id': 'int32','date': 'object', 'store_nbr': 'int32', 
                           'item_nbr': 'int32', 'onpromotion': 'bool'})
test = prepare_features(test)
test = test.set_index(['store_nbr', 'item_nbr', 'date'])

In [ ]:
from  sklearn.preprocessing import LabelEncoder
train_test_features = pd.concat([train_17, test], axis = 0)

cate_features = ['city','state','type', 'cluster', 'family']
noncate_features = ['month','day', 'dayofweek','onpromotion', 'class', 'perishable', 'type_h', 'oil_price_log_norm']
data_cate = train_test_features[cate_features]
data_noncate = train_test_features[noncate_features]
#get onehot vector for category variables
#data_cate = pd.get_dummies(data_cate, sparse = True)
#just encode category variables (except class) with normalized int

encoder = LabelEncoder()
for cate_col in cate_features:
    data_cate[cate_col] = encoder.fit_transform(data_cate[cate_col].astype('str'))

fullset_features = pd.concat([data_noncate, data_cate], axis = 1,
                             join_axes=[data_noncate.index])
obj_col = ['month', 'day', 'dayofweek', 'class']
fullset_features[obj_col] = fullset_features[obj_col].astype('int16')

#fullset_features.head(3)

In [ ]:
y_train = train_17['unit_sales']
X_train = fullset_features.iloc[:train_17['unit_sales'].shape[0]]
X_test = fullset_features.iloc[train_17['unit_sales'].shape[0]:]
print (fullset_features.shape, X_train.shape, y_train.shape, X_test.shape)
del fullset_features

In [ ]:
def check_coverage(df1, df2, col):    
    listA = sorted(df1[col].unique())
    listB = sorted(df2[col].unique())
    if listA == listB:
        print ('All the members in df2 are included in df1.')
    else:
        print ('Not all the members in df2 are included in df1.')
        A = set(df1[col].unique())
        B = set(df2[col].unique())
        print ('there is {:.5}% new {} seen in df2 \n'. format(((B-A)/len(A)) * 100, col)) 

In [ ]:
X_train_july = X_train.loc[(X_train['month'] == 7) & (X_train['day'] > 15)]
X_val = X_train.loc[(X_train['month'] == 8)]
y_train_july = y_train.iloc[:X_train_july.shape[0]]
y_val = y_train.iloc[-X_val.shape[0]:]
assert X_train_july.shape[0] == y_train_july.shape[0]
assert X_val.shape[0] == y_val.shape[0]

In [ ]:
def NWRMSLE(y, pred, weights_vector):
    weights = None
    y = y.clip(0, y.max())
    pred = pred.clip(0, pred.max())
    weights = np.where(weights_vector == 1, 1.25, 1.0) 
    score = np.nansum(weights * ((pred - y) ** 2)) / weights.sum()
    return np.sqrt(score)

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from time import time
def train_predict(learner, X_train, y_train, X_val, y_val): 
    results = {}    
    learner.fit(X_train, y_train)    
    print (learner.feature_importances_)       
        
    pred_train = learner.predict(X_train)
    pred_val = learner.predict(X_val)
    
    results['train_score'] = NWRMSLE(y_train, pred_train, X_train.perishable.values)
    results['val_score'] = NWRMSLE(y_val, pred_val, X_val.perishable.values)    
    
    print ('Data trained on {}'.format(learner.__class__.__name__))
    print ('traing and evaluation scores {},{}'.format(results['train_score'], results['val_score']))  
    return results

learner = RandomForestRegressor(n_estimators=500, max_features='sqrt', n_jobs=2)
train_predict(learner, X_train_july, y_train_july, X_val_aug, y_val_aug)

### Statistical features

In [6]:
# read selected time period of training data to generate statistical features 
train_17 = train_1617.loc[train_1617['date'] >= parse('2017-01-01')]
# load test data here
test = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/test.csv', parse_dates = ['date'],
                  dtype = {'id': 'int32','date': 'object', 'store_nbr': 'int32', 
                           'item_nbr': 'int32', 'onpromotion': 'bool'})
# cast the day-by-day onpromotion (T/F) value for each particular store * item
promo_train = train_17.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

promo_test = test.set_index(['store_nbr', 'item_nbr', 'date'])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)

#reindex and fillna for text data because there will be new items seen in text but not in train so na is generated
promo_test = promo_test.reindex(promo_train.index).fillna(False)
promo = pd.concat([promo_train, promo_test], axis=1)


In [7]:
sales_train = train_17.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
sales_train.columns = sales_train.columns.get_level_values(1)    

In [13]:
def get_timespan(df, st, minus, periods, freq='D'):
    return df[pd.date_range(st - timedelta(days=minus), periods=periods, freq=freq)]

def get_nearwd(date,b_date):
    date_list = pd.date_range(date-timedelta(140),periods=21,freq='7D').date
    result = date_list[date_list<=b_date][-1]
    return result

def prepare_dataset(t2017, is_train = True):
    X = pd.DataFrame({
        #if previous 14 days have any promo
        "promo_14days": get_timespan(promo, t2017, 14, 14).sum(axis=1).values,         
        "promo_60days": get_timespan(promo, t2017, 60, 60).sum(axis=1).values,
        "promo_140days": get_timespan(promo, t2017, 140, 140).sum(axis=1).values,
        #how many days in 14 days after not having any promotion
        "unpromo_14daysAfter":
        (1-get_timespan(promo, t2017+timedelta(14), 14, 14)).iloc[:,1:].sum(axis=1).values, 
    }, index = promo.index) #excluding the day itself so .iloc[:,1:]
    
    for i in range(7):
        X["promo_{}".format(i)] = promo[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
        for j in [14,60,140]:
            X["aft_promo_{}{}".format(i,j)] = (promo[t2017 + timedelta(days=i)]-1).values.astype(np.uint8)
            X["aft_promo_{}{}".format(i,j)] = X["aft_promo_{}{}".format(i,j)]*X['promo_{}'.format(i)]
     
    for i in range(7):
        #X['mean_4_dow{}'.format(i)] = get_timespan(sales_train, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['mean_12_dow{}'.format(i)] = get_timespan(sales_train, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        #X['mean_20_dow{}'.format(i)] = get_timespan(sales_train, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
        date = get_nearwd(t2017+timedelta(i),t2017)
        #ahead = (t2017-date).days
        #if ahead!=0:
        #    X['ahead0_{}'.format(i)] = get_timespan(sales_train, date+timedelta(ahead), ahead, ahead).mean(axis=1).values
        #    X['ahead7_{}'.format(i)] = get_timespan(sales_train, date+timedelta(ahead), ahead+7, ahead+7).mean(axis=1).values
        #X["day_1_{}1".format(i)]= get_timespan(sales_train, date, 1, 1).values.ravel()
        #X["day_1_{}2".format(i)]= get_timespan(sales_train, date-timedelta(7), 1, 1).values.ravel()
        for m in [3,7,14,30,60,140]:
            X["mean_{}_{}1".format(m,i)]= get_timespan(sales_train, date, m, m).\
                mean(axis=1).values
            X["mean_{}_{}2".format(m,i)]= get_timespan(sales_train, date-timedelta(7), m, m).\
                mean(axis=1).values
    if is_train:
        y = sales_train[pd.date_range(t2017, periods=16)].values
        return X, y
    return X



In [14]:
print("Preparing dataset...")

t2017 = date(2017,7,15)
X_l, y_l = [], []
#get train_stat features calculated based on duration of (7/15 - 7/15 + 14 days)
for i in range(3):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)  
    y_l.append(y_tmp)
X_train_stat = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0) # there will be 16 models for 16 date: july or aug 16-31

del X_l, y_l
t2017_val = date(2017, 7, 31)
X_val_stat, y_val = prepare_dataset(t2017_val) 
X_test_stat = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [15]:
print (X_train_stat.shape, y_train.shape, X_val_stat.shape, y_val.shape, X_test_stat.shape)

(502545, 116) (502545, 16) (167515, 116) (167515, 16) (167515, 116)


In [ ]:
del promo, sales_train, train_17, train_1617 

In [22]:
from sklearn.preprocessing import LabelEncoder
items['perishable'] = items['perishable'].astype('bool').fillna('False')
encoder = LabelEncoder()
col_toEncode = ['city','state','type', 'cluster'] 
for col in col_toEncode:
    stores[col] = encoder.fit_transform(stores[col].astype('str'))
col_toEncode = ['family', 'class']
for col in col_toEncode:
    items[col] = encoder.fit_transform(items[col].astype('str'))

In [27]:
stores.keys()

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')

In [39]:
def get_item_store_features(df):    
    result = df.reset_index().join(stores.set_index('store_nbr'), on = 'store_nbr')
    result = result.reset_index().join(items.set_index('item_nbr'), on = 'item_nbr')
    result = result.set_index(['store_nbr', 'item_nbr'])
    result = result.drop(['index'], axis = 1)
    return result

X_train = get_item_store_features(X_train_stat)
X_val = get_item_store_features(X_val_stat)
X_test = get_item_store_features(X_test_stat)

In [38]:
X_train.head()

promo_140days  promo_14days  promo_60days  \
store_nbr item_nbr                                              
1         96995                 0             0             0   
          99197                 0             0             0   
          103520                0             0             0   
          103665                0             0             0   
          105574               30             0             0   

                    unpromo_14daysAfter  promo_0  aft_promo_014  \
store_nbr item_nbr                                                
1         96995                      13        0              0   
          99197                      13        0              0   
          103520                     13        0              0   
          103665                     13        0              0   
          105574                     13        0              0   

                    aft_promo_060  aft_promo_0140  promo_1  aft_promo_114  \
store_nbr item_nbr                                                          
1         96995                 0               0        0              0   
          99197                 0               0        0              0   
          103520                0               0        0              0   
          103665                0               0        0              0   
          105574                0               0        0              0   

                       ...      mean_60_62  mean_140_61  mean_140_62  city  \
store_nbr item_nbr     ...                                                   
1         96995        ...        0.127077     0.094911     0.094911    18   
          99197        ...        0.618532     0.287785     0.265085    18   
          103520       ...        0.843241     0.806579     0.783879    18   
          103665       ...        1.070664     1.025455     1.031856    18   
          105574       ...        1.716859     1.781987     1.773880    18   

                    state  type  cluster  family  class  perishable  
store_nbr item_nbr                                                   
1         96995        12     3        4      12     64       False  
          99197        12     3        4      12     44       False  
          103520       12     3        4      12     17       False  
          103665       12     3        4       5    187        True  
          105574       12     3        4      12     31       False  

[5 rows x 123 columns]

In [ ]:
del X_train_stat, X_val_stat, X_test_stat

In [44]:
def NWRMSLE(y, pred, weights_vector):
    weights = None
    y = y.clip(0, y.max())
    pred = pred.clip(0, pred.max())
    weights = np.where(weights_vector == 1, 1.25, 1.0) 
    score = np.nansum(weights * ((pred - y) ** 2)) / weights.sum()
    return np.sqrt(score)

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from time import time
def train_predict(learner, X_train, y_train, X_val, y_val): 
    results = {}    
    learner.fit(X_train, y_train)    
    print (learner.feature_importances_)       
        
    pred_train = learner.predict(X_train)
    pred_val = learner.predict(X_val)
    
    results['train_score'] = NWRMSLE(y_train, pred_train, X_train.perishable.values)
    results['val_score'] = NWRMSLE(y_val, pred_val, X_val.perishable.values)    
    
    print ('Data trained on {}'.format(learner.__class__.__name__))
    print ('traing and evaluation scores {},{}'.format(results['train_score'], results['val_score']))  
    return results

learner = RandomForestRegressor(n_estimators=500, max_features='sqrt', n_jobs=2)
train_predict(learner, X_train, y_train[:,0].reshape(X_train.shape[0]), 
              X_val, y_val[:,0].reshape(X_val.shape[0]))

[ 0.00254285  0.00184126  0.00231792  0.00209733  0.00684065  0.          0.
  0.          0.00108641  0.          0.          0.          0.00090381
  0.          0.          0.          0.00052063  0.          0.          0.
  0.00048723  0.          0.          0.          0.00046105  0.          0.
  0.          0.00036352  0.          0.          0.          0.02504953
  0.00300765  0.07035944  0.00711245  0.06596959  0.0088748   0.05307327
  0.00822968  0.01333634  0.00378787  0.00359839  0.00281756  0.00413083
  0.00328941  0.00393313  0.0031399   0.01073394  0.00357393  0.01225143
  0.00443816  0.00412681  0.00336567  0.00284227  0.00298071  0.00955587
  0.00360406  0.01060357  0.00309643  0.01781069  0.00323823  0.01648382
  0.00350636  0.00585065  0.00360737  0.00286884  0.00290016  0.00587725
  0.0033014   0.01836175  0.00322638  0.02866302  0.00323848  0.01398794
  0.00328032  0.00525684  0.00419702  0.00286623  0.0028921   0.0038273
  0.00307479  0.02878804  0.00312458  0.

{'train_score': 0.21075555484650327, 'val_score': 0.5802546367086151}

In [45]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
def NWRMSLE(y, pred, weights_vector):
    weights = None
    y = y.clip(0, y.max())
    pred = pred.clip(0, pred.max())
    weights = np.where(weights_vector == 1, 1.25, 1.0) 
    score = np.nansum(weights * ((pred - y) ** 2)) / weights.sum()
    return np.sqrt(score)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train[:,0].reshape(X_train.shape[0]))
                       #weight=X_train.perishable.values * 0.25 + 1)
lgb_val = lgb.Dataset(X_val, y_val[:,0].reshape(X_val.shape[0]), reference=lgb_train)
                     #weight=X_val.perishable.values * 0.25 + 1)

MAX_ROUNDS = 1000
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2_root',
    'num_leaves': 31,
    'lambda_l1':0.1,
    'num_data_in_leaf':200,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_threads': 4
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=MAX_ROUNDS,
                valid_sets= lgb_val,
                early_stopping_rounds=50, 
                verbose_eval=100
                )
print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))

print('Start predicting...')
# predict
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration or MAX_ROUNDS)
# eval
print('The mrse of prediction is:', mean_squared_error(y_val[:,0], y_pred)**0.5)

Start training...
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's rmse: 0.57736
Early stopping, best iteration is:
[103]	valid_0's rmse: 0.577317
mean_14_01: 551746.71
mean_30_01: 540393.32
mean_30_61: 278660.22
mean_60_01: 95525.33
mean_7_01: 89903.32
mean_3_31: 58112.84
mean_3_32: 56403.16
mean_60_41: 48592.61
unpromo_14daysAfter: 36215.65
mean_60_51: 25777.10
mean_140_01: 18647.60
class: 17566.43
promo_14days: 14503.54
type: 11246.42
state: 10301.84
mean_3_01: 9491.77
mean_3_41: 6865.55
city: 6683.73
family: 6614.17
cluster: 6290.82
mean_140_12: 6233.14
mean_3_22: 5568.01
mean_30_31: 5265.93
mean_30_32: 5095.61
promo_60days: 4663.32
mean_140_22: 4216.51
promo_140days: 3906.25
mean_3_42: 3641.36
mean_60_31: 3454.44
mean_7_12: 3408.74
mean_30_22: 3256.57
mean_140_61: 2971.09
mean_60_22: 2810.89
mean_60_21: 2809.77
mean_3_12: 2705.44
mean_3_52: 2460.70
mean_3_21: 2443.89
mean_60_12: 2306.76
mean_30_21: 2263.91
mean_3_11: 2251.76
mean_3_51: 2170.12
mean_3_6

In [41]:
val_pred = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, y_train[:,i].reshape(X_train.shape[0]),
        weight=X_train["perishable"] * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, y_val[:,i].reshape(X_val.shape[0]), reference=lgb_train, 
        weight=X_val.perishable.values * 0.25 + 1
        )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    
assert y_val.shape == np.array(val_pred).transpose().shape
print("Validation msre:", mean_squared_error(y_val, np.array(val_pred).transpose()))

Step 1
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 0.5554	valid_1's rmse: 0.579688
Early stopping, best iteration is:
[89]	training's rmse: 0.556544	valid_1's rmse: 0.579505
mean_14_01: 884882.33
mean_7_01: 841727.64
mean_3_01: 115444.36
mean_30_01: 29978.77
promo_0: 27702.00
mean_3_21: 19345.53
mean_60_01: 11713.11
promo_14days: 11688.34
mean_3_22: 11210.60
mean_60_21: 6128.22
mean_140_01: 5023.23
mean_60_22: 4009.32
mean_30_11: 3941.00
mean_3_51: 3683.19
promo_2: 3067.40
unpromo_14daysAfter: 2900.65
mean_3_11: 2584.44
mean_140_12: 2541.38
mean_60_31: 2105.45
mean_3_61: 2092.09
mean_140_22: 1999.34
promo_60days: 1808.75
mean_30_21: 1806.62
class: 1732.44
type: 1685.20
city: 1433.22
cluster: 1368.83
mean_140_32: 1368.66
family: 1350.31
mean_30_22: 1324.99
mean_3_41: 1247.51
state: 1226.04
mean_3_32: 1098.26
mean_30_12: 1092.58
mean_60_12: 925.64
mean_60_32: 901.09
mean_7_61: 840.12
mean_7_51: 821.35
promo_140days: 814.84
mean_60_42: 733.94
mean_

[100]	training's rmse: 0.582738	valid_1's rmse: 0.597572
Early stopping, best iteration is:
[53]	training's rmse: 0.58904	valid_1's rmse: 0.594824
mean_14_01: 719618.08
mean_7_01: 519210.57
mean_30_01: 298698.67
promo_3: 51008.65
mean_3_01: 26700.32
mean_3_61: 14752.18
mean_60_01: 13697.27
promo_14days: 12414.29
mean_30_61: 10505.43
mean_140_01: 6642.96
unpromo_14daysAfter: 4957.40
mean_30_41: 4512.31
mean_3_62: 4074.88
mean_3_51: 3577.24
mean_60_61: 3554.97
type: 3228.02
promo_0: 2938.89
mean_140_32: 2868.73
mean_140_12: 2687.58
promo_4: 2548.79
family: 2514.75
mean_30_42: 2359.14
mean_140_22: 2111.43
mean_3_41: 1986.35
class: 1934.52
mean_14_12: 1681.23
promo_60days: 1378.88
mean_60_52: 1351.12
mean_60_12: 1291.79
mean_60_42: 1290.39
mean_60_41: 1256.04
mean_30_52: 1213.93
mean_30_51: 1110.93
mean_3_52: 995.39
mean_60_22: 898.07
city: 843.75
mean_140_42: 771.81
mean_3_31: 757.52
mean_3_42: 731.40
mean_60_51: 677.85
mean_3_21: 662.02
mean_60_62: 656.36
mean_30_62: 651.50
mean_7_51: 64

Step 8
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[42]	training's rmse: 0.61145	valid_1's rmse: 0.626622
mean_30_01: 699228.48
mean_7_01: 491181.59
mean_14_01: 466677.22
mean_3_01: 39456.90
unpromo_14daysAfter: 37744.29
mean_60_01: 36741.81
mean_3_21: 20282.27
mean_60_21: 11370.21
mean_60_31: 10327.89
promo_14days: 10314.47
mean_140_01: 8104.57
mean_3_22: 7496.57
mean_60_22: 4970.09
mean_140_12: 4062.79
mean_140_22: 3168.49
type: 2972.30
promo_60days: 2782.33
mean_30_21: 2346.86
mean_60_12: 2280.28
mean_3_31: 2143.50
mean_140_61: 1886.40
class: 1866.63
family: 1607.73
city: 1386.02
mean_60_11: 1348.10
promo_6: 1327.04
mean_60_61: 1267.65
mean_30_12: 1157.62
mean_3_41: 1064.73
perishable: 992.07
cluster: 968.21
mean_30_61: 894.24
mean_3_11: 848.11
mean_140_21: 845.41
mean_140_32: 750.60
promo_140days: 672.87
promo_2: 628.01
state: 596.68
mean_60_32: 457.48
mean_3_51: 456.97
promo_1: 449.18
mean_3_12: 439.01
mean_7_02: 433.01
mean_14_

Step 11
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 0.608226	valid_1's rmse: 0.62683
[200]	training's rmse: 0.599384	valid_1's rmse: 0.62534
[300]	training's rmse: 0.592885	valid_1's rmse: 0.624948
[400]	training's rmse: 0.587374	valid_1's rmse: 0.624461
Did not meet early stopping. Best iteration is:
[400]	training's rmse: 0.587374	valid_1's rmse: 0.624461
mean_30_01: 562665.86
mean_7_01: 405200.61
mean_14_01: 384909.06
unpromo_14daysAfter: 48079.98
mean_30_61: 47383.02
mean_60_01: 34389.26
mean_60_61: 31842.76
mean_3_61: 26516.80
mean_3_01: 15480.92
mean_7_12: 15083.29
promo_14days: 12209.70
class: 11544.45
family: 10058.42
mean_140_01: 8319.19
mean_3_41: 8128.88
promo_3: 8080.55
mean_60_51: 6636.21
type: 6394.86
mean_14_52: 5221.85
mean_30_41: 5200.24
promo_60days: 5154.54
mean_140_12: 4701.66
mean_14_02: 3143.05
mean_140_32: 3018.34
mean_30_51: 2845.23
promo_4: 2749.30
promo_140days: 2727.68
mean_3_51: 2679.67
mean_3_22: 2317.71
mean_140_42:

Step 14
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 0.611259	valid_1's rmse: 0.61877
[200]	training's rmse: 0.601133	valid_1's rmse: 0.61626
[300]	training's rmse: 0.594374	valid_1's rmse: 0.615258
[400]	training's rmse: 0.588439	valid_1's rmse: 0.615052
Did not meet early stopping. Best iteration is:
[400]	training's rmse: 0.588439	valid_1's rmse: 0.615052
mean_30_01: 840763.31
mean_30_61: 247440.26
mean_14_01: 163092.56
mean_7_01: 108288.79
mean_3_01: 107338.08
mean_60_01: 62405.74
unpromo_14daysAfter: 42126.16
promo_14days: 17868.68
mean_3_12: 16826.78
class: 15251.85
mean_3_11: 13365.69
promo_6: 12526.49
mean_3_21: 10176.83
family: 7405.52
mean_140_01: 7370.56
type: 6187.02
mean_60_21: 5877.37
mean_140_12: 5798.35
promo_140days: 4964.81
promo_60days: 4960.16
city: 4480.47
mean_140_22: 4281.20
mean_60_12: 3478.09
mean_3_61: 2837.49
mean_60_11: 2813.82
cluster: 2628.05
mean_3_31: 2624.87
mean_3_51: 2471.05
mean_60_22: 2426.87
mean_30_11: 2412.

Validation msre: 0.378200033302


In [42]:
print("Validation mse:", mean_squared_error(y_val, np.array(val_pred).transpose()))
weight = X_val["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))


Validation mse: 0.378200033302
nwrmsle = 0.6156949436893447


In [ ]:
X_train.keys()